In [1]:
import numpy as np
import pandas as pd
import spacy
from spacy.tokens import Doc, Token,Span
from spacy.language import Language
import json
import requests
import os
from dotenv import load_dotenv
import requests


In [ ]:
#not happy with this one ...
nlp = spacy.load("pl_core_news_sm")
import pl_core_news_sm
nlp = pl_core_news_sm.load()
nlp_clean=pl_core_news_sm.load()
global ent_list 
ent_list=[]
with open('ents.json',  mode='r', encoding='utf-8') as file:
    file.seek(0)
    data = json.load(file)
    
    @Language.component('recognize_ents')
    def RecoginzeEnts(doc):
       
        print(type(doc))
       
        for ent in doc.ents:
            print(ent.text , ent.label_)
            ent_list.append(ent.text)
        return doc
    def ReadAndCheck(df):
        for item in [f'nlp_{i}' for i in range(2, 6)]:
       
            text_data = df[item].apply(lambda x:nlp(x))
            # RecoginzeEnts(text_data)
            data["ents"]["data"].extend(ent_list)
            ent_list.clear()
            # print(f"{text_data}_recieved")
        return df
    nlp.add_pipe('recognize_ents', after='ner')
    DATASET_CHECK='NLP_CONNECTED.xlsx'
    df = pd.read_excel(io=DATASET_CHECK)
    df=ReadAndCheck(df)
    # file.seek(0)
    with open('ents.json', 'w') as f:
        # for item in data:
        #     print(type(data))
        data["ents"]["data"]=list(set(data["ents"]["data"]))
        json.dump(data, f)
 


In [ ]:
##Getting the games 
from igdb.wrapper import IGDBWrapper
load_dotenv()
client_secret=os.getenv('CLIENT_SECRET')
client_id=os.getenv('CLIENT_ID')
print(client_id)
x = requests.post(f'https://id.twitch.tv/oauth2/token?client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials')
response= json.loads(x.text)
# x = requests.post('https://id.twitch.tv/oauth2/token?client_id=sdd7tjwylco6cld0pxk6rzatuaj0hu&client_secret=ns464kuoebfh8l1kvrujnikywuteo9&grant_type=client_credentials')
# print(x.text)
wrapper = IGDBWrapper(f"{client_id}", response["access_token"])
byte_array = wrapper.api_request(
            'games',
            'fields name,total_rating_count; sort total_rating_count desc;where total_rating_count!= null; offset 0; where platforms=48;'
          )
games=json.loads(byte_array)
print(games)
ent_list=[]
with open('ents.json',  mode='r', encoding='utf-8') as file:
    file.seek(0)
    data = json.load(file)
    
    @Language.component('recognize_ents')
    def RecoginzeEnts(doc):
       
        print(type(doc))
       
        for ent in doc.ents:
            print(ent.text , ent.label_)
            ent_list.append(ent.text)
        return doc
    def EasyRead(array):
        for item in array:
            text_data=nlp(item['name'])
            data["ents"]["data"].extend(ent_list)
            ent_list.clear()
        return array
    def ReadAndCheck(df):
        for item in ['Title']:
       
            text_data = df[item].apply(lambda x:nlp(x))
            # RecoginzeEnts(text_data)
            data["ents"]["data"].extend(ent_list)
            ent_list.clear()
            # print(f"{text_data}_recieved")
        return df
    
    DATASET_CHECK='MostPopularGames.xlsx'
    df = pd.read_excel(io=DATASET_CHECK)
    df=ReadAndCheck(df)
    array=EasyRead(games)
    # file.seek(0)
    with open('ents.json', 'w') as f:
        # for item in data:
        #     print(type(data))
        data["ents"]["data"]=list(set(data["ents"]["data"]))
        json.dump(data, f)
 

In [ ]:
from bs4 import BeautifulSoup

with open("gamer_slang.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')
    div_tag = soup.find("div", {"class": "slang-container"})
    li_tags = div_tag.find_all("li")
    # for tag in li_tags:
    #     print(tag)
    ent_list=[]
    with open('ents.json',  mode='r', encoding='utf-8') as file:
        file.seek(0)
        data = json.load(file)
        def ReadAndCheck(li_tags):
            for tag in li_tags:
                strong_tag = tag.find("strong")
                if strong_tag:
                    ent_list.append(strong_tag.text)
                    data["ents"]["data"].extend(ent_list)
                    ent_list.clear()
            # print(tag)
        # for item in ['Title']:
       
        #     text_data = df[item].apply(lambda x:nlp(x))
        #     # RecoginzeEnts(text_data)
        #     data["ents"]["data"].extend(ent_list)
        #     ent_list.clear()
        #     # print(f"{text_data}_recieved")
            return li_tags
        li_tags=ReadAndCheck(li_tags)
       
    # file.seek(0)
    with open('ents.json', 'w') as f:
        # for item in data:
        #     print(type(data))
        data["ents"]["data"]=list(set(data["ents"]["data"]))
        json.dump(data, f)

In [6]:
from pathlib import Path
from spacy.pipeline import EntityRuler
from spacy.lang.pl import Polish
import inspect
# def foo():
#     print('hello')
# print(inspect.getsource(foo))
# inspect.getfile(foo)

# nlp = spacy.load("pl_core_news_sm")
# import pl_core_news_sm
# nlp = pl_core_news_sm.load()
# @Language.factory('custom_ruler')
# class CustomEntityRuler(EntityRuler):
#     def __init__(self, nlp, name=None, overwrite_ents=False, validate=True, **cfg):
#         EntityRuler.__init__(self, nlp, overwrite_ents=overwrite_ents, validate=validate, **cfg)
#         self.name = name if name is not None else "custom_ruler"
#
#     def add_patterns(self, patterns):
#         super().add_patterns(patterns)
# def custom_ruler_factory(nlp):
#     return CustomEntityRuler(nlp)
def create_training_data(f, type):
    with open('ents.json',  mode='r', encoding='utf-8') as file:
        file.seek(0)
        data = json.load(file)
        patterns=[]
        for  item in data["ents"]["data"]:
            pattern={
                "label":type,
                "pattern":item
            }
            patterns.append(pattern)

        # pattern_regex={"label": "GAMER", "pattern": [{"REGEX": r"\b(?:[Gg]amer|[Pp]layer)\b"}]}
        # patterns.append(pattern_regex)
        return (patterns)
def generate_rules(patterns):
    nlp = Polish()
    # ruler = CustomEntityRuler(nlp, name="instance_ruler")
    # ruler.add_patterns(patterns)
    ruler=EntityRuler(nlp, 'new_ruler')
    # ner=nlp.get_pipe('entity_ruler')
    ruler.add_patterns(patterns)
    nlp.add_pipe( 'entity_ruler', name="new_ruler")
    nlp.to_disk('gg_ner')
# label_tuples = [(25, 30, 'GAMER')]
# label_tuple = tuple(label_tuples)
patterns=create_training_data('ready.json', "GAMER") 
print(patterns)
generate_rules(patterns)
nlp2 = spacy.load('gg_ner')

# Extract labels from the loaded model's EntityRuler
entity_ruler = nlp2.get_pipe('new_ruler')
entity_ruler.add_patterns(patterns);
labels = {}
for pattern in entity_ruler.patterns:
    label = pattern['label']
    if label not in labels:
        labels[label] = [pattern["pattern"]]
    else:
        labels[label].append(pattern["pattern"])

# Update the 'meta.json' file with the new labels
nlp2.meta["labels"]["new_ruler"] = labels

# Save the model again
nlp2.to_disk('gg_ner')
# meta_path = Path('gg_ner') / 'meta.json'
# with meta_path.open('r', encoding='utf-8') as file:
#     meta = json.load(file)
# meta['labels']['entity_ruler'] = "GAMER"
# with meta_path.open('w', encoding='utf-8') as file:
#     json.dump(meta, file)
# with open('ready.json', 'w') as f:
#         # for item in data:
#         #     print(type(data))
#     data["ents"]["data"]=list(set(patterns))
#     json.dump(patterns, f)

[{'label': 'GAMER', 'pattern': 'Shooter'}, {'label': 'GAMER', 'pattern': 'level'}, {'label': 'GAMER', 'pattern': 'levele'}, {'label': 'GAMER', 'pattern': 'levelu'}, {'label': 'GAMER', 'pattern': 'leveli'}, {'label': 'GAMER', 'pattern': 'funu'}, {'label': 'GAMER', 'pattern': 'fun'}, {'label': 'GAMER', 'pattern': 'dla funu'}, {'label': 'GAMER', 'pattern': 'Top eleven'}, {'label': 'GAMER', 'pattern': 'Coin masters'}, {'label': 'GAMER', 'pattern': 'PvE'}, {'label': 'GAMER', 'pattern': 'player versus environment'}, {'label': 'GAMER', 'pattern': 'IDC'}, {'label': 'GAMER', 'pattern': 'Ai'}, {'label': 'GAMER', 'pattern': 'Artificial Intelligence'}, {'label': 'GAMER', 'pattern': 'player'}, {'label': 'GAMER', 'pattern': 'Melee'}, {'label': 'GAMER', 'pattern': 'Hardware'}, {'label': 'GAMER', 'pattern': 'Battle royale '}, {'label': 'GAMER', 'pattern': 'Random'}, {'label': 'GAMER', 'pattern': 'Mario World'}, {'label': 'GAMER', 'pattern': 'AOE'}, {'label': 'GAMER', 'pattern': ' Area of Effect '}, {'

In [12]:
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example


def create_ner_training_data_with_negatives(file_path, label, negative_samples):
    with open(file_path, mode='r', encoding='utf-8') as file:
        data = json.load(file)
        training_data = []
        for item in data["ents"]["data"]:
            training_data.append((item, {"entities": [(0, len(item), label)]}))

        # Add negative samples
        for item in negative_samples:
            training_data.append((item, {"entities": []}))
    return training_data

negative_samples = ["Hello", "Goodbye", "Example sentence", "Another example", "noc", "zdjęcia", "zabawa", "spać", "nosić", "lubić", "cześć"]
ner_training_data = create_ner_training_data_with_negatives('ents.json', "GAMER", negative_samples)
random.shuffle(ner_training_data)
print(ner_training_data[:5])
def train_ner_model(training_data, n_iter=100, model=None, output_dir=None, new_model_name="custom_ner"):
    if model is not None:
        nlp = spacy.load(model)
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("pl")
        print("Created blank 'pl' model")

    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner", last=True)
    else:
        ner = nlp.get_pipe("ner")

    # for _, annotations in training_data:
    #     for ent in annotations.get("entities"):
    #         ner.add_label(ent[2])
    examples = []
    for text, annotations in training_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        examples.append(example)

    nlp.initialize(lambda: examples)

    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()

    for i in range(30):
        losses = {}
    # Split examples into smaller batches using minibatch function
    for batch in minibatch(examples, size=compounding(4.0, 32.0, 1.001)):
        nlp.update(batch, losses=losses)
    print("Iteration:", i, "Losses:", losses)

    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)
def test_model(model, text):
    doc = model(text)
    for ent in doc.ents:
        print(ent.text, ent.label_)
train_ner_model(ner_training_data, n_iter=20, output_dir="gg_ner")
nlp2 = spacy.load("gg_ner")
sample_text = "Cześć"
test_model(nlp2, sample_text)

[('Bot ', {'entities': [(0, 4, 'GAMER')]}), ('Bug', {'entities': [(0, 3, 'GAMER')]}), ('Rozwesela', {'entities': [(0, 9, 'GAMER')]}), ('Pokémon Sun', {'entities': [(0, 11, 'GAMER')]}), ('Trick', {'entities': [(0, 5, 'GAMER')]})]
Created blank 'pl' model
Iteration: 29 Losses: {'ner': 232.88672876129542}
Saved model to gg_ner
Cześć GAMER


In [ ]:
nlp=spacy.load("gg_ner")
nlp('player')
# def test_model(model, text):
#     doc = model(text)
#     for ent in doc.ents:
#         print(ent.text, ent.label_)
# with open('ents.json', 'r') as file:
#     file.seek(0)
#     data = json.load(file)
#     hits=[]
#     for item in data["ents"]["data"]:
#         # print(item)
#         resluts=test_model(nlp,item)
    #     for result in resluts:
    #         hits.append(result)
    # print(hits)




Shooter GAMER
level GAMER
levele GAMER
levelu GAMER
PvE, player versus environment GAMER
IDC GAMER
Ai GAMER
Artificial Intelligence GAMER
Melee GAMER
Hardware GAMER
Battle royale GAMER
Random GAMER
Mario World GAMER
AOE GAMER
 Area of Effect GAMER
Trial GAMER
FAQ GAMER
HP GAMER
 health points GAMER
Lama GAMER
Raid GAMER
Złota edycja GAMER
F2F, face to face GAMER
QS GAMER
quickshot GAMER
Intro GAMER
Early Access GAMER
Rendering GAMER
Respawn GAMER
Manual GAMER
Emerald GAMER
Potki GAMER
potiony GAMER
POV GAMER
God mode GAMER
Screenshot GAMER
OMG GAMER
FPS GAMER
first person shooter GAMER
TDM GAMER
team deathmatch GAMER
Upgrade GAMER
Evan GAMER
New Horizons GAMER
Trafienie krytyczne GAMER
crit GAMER
krit GAMER
Fifa16 GAMER
DMG GAMER
damage GAMER
bede GAMER
bd GAMER
DPS GAMER
 damage per second GAMER
BB GAMER
Humor GAMER
Remaster GAMER
Ult GAMER
 ultimate GAMER
HF GAMER
have fun GAMER
Marnotrawstwo GAMER
IRL GAMER
In real life GAMER
Imersja GAMER
Main GAMER
Little GAMER
PvP GAMER
player ve

In [1]:
nlp=spacy.load("gg_ner")
nlp('player')

NameError: name 'spacy' is not defined